# Объединяем датасеты реального человека и бота

In [113]:
import pandas as pd
import ast
from tqdm.notebook import tqdm

In [114]:
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub
import numpy as np

In [2]:
vlakhov_df = pd.read_csv('/Users/kseniapetuhova/Desktop/pd_vlakhov_2.csv')

In [4]:
lava_df = pd.read_csv('/Users/kseniapetuhova/Desktop/pd_lava.csv')

In [6]:
del lava_df['Unnamed: 0']

In [7]:
del lava_df['Unnamed: 0.1']

In [10]:
del vlakhov_df['Unnamed: 0']

In [12]:
vlakhov_shalava = pd.concat([vlakhov_df, lava_df], ignore_index=True)

In [14]:
posts = list(vlakhov_shalava.posts)

In [35]:
import numpy as np

In [ ]:
posts_new = []
for post in tqdm(posts):
    if post != None and type(post) != float and post != [] and post != '[]':
        post = post.replace('\/', '')
        p = ast.literal_eval(post[1:-1])
        posts_new.append(p)
    else:
        posts_new.append(None)

In [79]:
texts = []
all_reposts = []
comments =[]
likes = []
reposts_count = []
for user in tqdm(posts_new):
    if user != None:
        if type(user) is tuple:
            texts_per_user = []
            reposts = []
            comm = []
            lik = []
            rep_c = []
            for post in user:
                text = post['text']
                texts_per_user.append(text)
                from_id = post['from_id']
                owner_id = post['owner_id']
                if from_id == owner_id:
                    reposts.append(0)
                else:
                    reposts.append(1)

                comm.append(post['comments']['count'])
                lik.append(post['likes']['count'])
                rep_c.append(post['reposts']['count'])
            texts.append(texts_per_user)
            all_reposts.append(reposts)
            comments.append(comm)
            likes.append(lik)
            reposts_count.append(rep_c)
        
        else:
            text = user['text']
            texts.append(text)
            from_id = user['from_id']
            owner_id = user['owner_id']
            if from_id == owner_id:
                all_reposts.append(0)
            else:
                all_reposts.append(1)

            comments.append(user['comments']['count'])
            likes.append(user['likes']['count'])
            reposts_count.append(user['reposts']['count'])

    else:
        texts.append(None)
        all_reposts.append(None)
        comments.append(None)
        likes.append(None)
        reposts_count.append(None)

In [83]:
vlakhov_shalava['texts'] = texts
vlakhov_shalava['repost'] = all_reposts
vlakhov_shalava['comments'] = comments
vlakhov_shalava['likes'] = likes
vlakhov_shalava['reposts_count'] = reposts_count

In [97]:
import tqdm.notebook as tqdm

In [86]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
use_model = hub.load(module_url)
print("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder-multilingual/3 loaded


In [100]:
def use_get_embedding(text, model):
    return model([text]).numpy()[0]

def use_records_to_embeds(list_text, model):
    embeddings = np.zeros((len(list_text), 512))
    for i in tqdm(range(len(list_text))):
        embeddings[i] = use_get_embedding(list_text[i], model)
    return embeddings

In [101]:
texts_to_embed = []
for user in tqdm(texts):
    if user != None:
        text_new = ' '.join(user)
        text_new = text_new.replace('\n', ' ')
        texts_to_embed.append(text_new)
        
    else:
        texts_to_embed.append('')

In [103]:
use_embeddings_targets = use_records_to_embeds(texts_to_embed, use_model)

In [104]:
list_use_embeddings_targets = []
for embed in use_embeddings_targets:
    new_embed = embed.tolist()
    list_use_embeddings_targets.append(new_embed)

In [105]:
vlakhov_shalava['embeddings_texts'] = list_use_embeddings_targets

In [116]:
vlakhov_shalava = pd.read_csv('vlakhov_shalava.csv')

In [132]:
comments = list(vlakhov_shalava.comments)


In [134]:
comments_new = []
for comment in comments:
    if type(comment) == str:
        new = ast.literal_eval(comment)
        comments_new.append(new)
    else:
        comments_new.append(None)

In [138]:
from statistics import mean

In [143]:
mean_comments = []
for comm in tqdm(comments_new):
    if comm != None and type(comm) != int:
        mean_com = mean(comm)
        mean_comments.append(mean_com)
    elif type(comm) == int:
        mean_comments.append(comm)
    else:
        mean_comments.append(None)

In [144]:
vlakhov_shalava['comments_mean'] = mean_comments

In [152]:
likes = list(vlakhov_shalava.reposts_count)

In [153]:
likes_new = []
for comment in likes:
    if type(comment) == str:
        new = ast.literal_eval(comment)
        likes_new.append(new)
    else:
        likes_new.append(None)

In [154]:
mean_likes = []
for comm in tqdm(likes_new):
    if comm != None and type(comm) != int:
        mean_com = mean(comm)
        mean_likes.append(mean_com)
    elif type(comm) == int:
        mean_likes.append(comm)
    else:
        mean_likes.append(None)

In [155]:
vlakhov_shalava['reposts_mean'] = mean_likes

In [157]:
list_use_embeddings = list(vlakhov_shalava.embeddings_texts)

In [158]:
list_use_embeddings_targets = []
for use in list_use_embeddings:
    new = ast.literal_eval(use)
    list_use_embeddings_targets.append(new)

In [159]:
columns_names2 = [i for i in range(0, 512)]
col_1024_df = pd.DataFrame(list_use_embeddings_targets, columns=columns_names2)
new_df = pd.concat([vlakhov_shalava, col_1024_df], axis=1)

In [161]:
new_df.to_csv('vlakhov_shalava_features.csv')